In [1]:
import derinet.lexicon as dlex
import os
lexicon = dlex.Lexicon()
current_dir = os.getcwd()  # aktualni adresar
file_path = os.path.join(current_dir, "./derinet-2-3.tsv")  #sestaveni cesty
lexicon.load(file_path)

testování 1.1: vyhledávání slov, které jsou stejná, ale jsou tam vícekrát (Adam, Adam)
    - uvést druh daného slova a jestli je osamocen nebo má nějakého rodiče (absurdno, které tam je dvakrát, jednou o samotě, jednou ve stromě)
testování 1.2: vyhledávání slov, které jsou stejná, liší se pouze tím, že je na začátku jiné písmeno velikostně (absurdistán, Absurdistán)
    - to samé

In [ ]:
#testování 1
def tisknuti(veta, soubor, seznam):

    # -------------------- výstup --------------------
    with open(f"{soubor}.txt", "w", encoding="utf-8") as f:
        f.write(f"{veta}\n")
        f.write(f"{'SLOVO 1'.ljust(20)}{'SLOVO 2'.ljust(20)}{'SLOVNÍ DRUH'.ljust(10)}\n")
        f.write("\n")

        zmena = "velké"
        for i in sorted(seznam, key=lambda x: x[0]):
            if i[0][0].islower() and zmena == "velké":
                f.write("\n")
                zmena = "malé"
            elif i[0][0].isupper() and zmena == "malé":
                f.write("\n")
                zmena = "velké"
            
            f.write(f"{i[0].ljust(20)}{i[1].ljust(20)}{i[2].ljust(10)}\n")


videno = set()
stejny_typ, ruzny_typ = set(), set()

for lexeme in lexicon.iter_lexemes():
    if lexeme not in videno: #pokud už jsem to slovo testoval, tak ho přeskočím
        dalsi = lexicon.get_lexemes(lemma=lexeme.lemma)
        if len(dalsi) > 1:
            dalsi.remove(lexeme) #takhle předejdu tomu, abych zpracoval stejné slovo dvakrát 
            videno.add(lexeme)
            for lex in dalsi:
                if lexeme.all_parents == [] and lex.all_parents == []:
                    if lexeme.pos != lex.pos: #pokud mají různé typy
                        ruzny_typ.add((lexeme.lemma, lex.lemma, lex.pos))
                        #f.write(f"{lexeme.lemma}, {lex.lemma} -  RŮZNÉ typy: {lexeme.pos}, {lex.pos} \n") #nejdříve původní, potom nalezlý
                    else:
                        stejny_typ.add((lexeme.lemma, lex.lemma, lexeme.pos, lex.pos))
                        #f.write(f"{lexeme.lemma}, {lex.lemma} - STEJNÝ typ: {lex.pos} \n")

tisknuti("TEST: IDENTICKÁ SLOVA, KTERÁ K SOBĚ NAVZÁJEM NEJSOU NIJAK PŘIPOJENA A MAJÍ STJENÝ SLOVNÍ DRUH","test1.1", stejny_typ)
tisknuti("TEST: IDENTICKÁ SLOVA, KTERÁ K SOBĚ NEJSOU PŘIPOJENA A NESDÍLÍ STEJNÝ SLOVNÍ DRUH","test1.2", ruzny_typ)




In [8]:
videno = set()
stejny_typ = set()
ruzny_typ = set()
            #ted 1.2 - různá velikost začínajících písmen
for lexeme in lexicon.iter_lexemes():
    if lexeme.lemma not in videno and not lexeme.lemma.startswith("-"):
        cast = lexeme.lemma[1:]
        pismeno = lexeme.lemma[0].swapcase()
        lexeme1 = pismeno + cast #přetvořím si to slovo tak, aby tady bylo na začátku obrácená velikost písmena
        for lex in lexicon.get_lexemes(lemma=lexeme1): #najdu jestli, tam je nějaké obrácené velikosti
            videno.add(lexeme)
            videno.add(lex)
            #oboje tam přidám
            if lexeme.all_parents == [] and lex.all_parents == []:
                if lexeme.pos != lex.pos: #pokud mají různé typy
                    ruzny_typ.add((lexeme.lemma, lex.lemma, lex.pos))
                    #f.write(f"{lexeme.lemma}, {lex.lemma} -  RŮZNÉ typy: {lexeme.pos}, {lex.pos} \n") #nejdříve původní, potom nalezlý
                else:
                    stejny_typ.add((lexeme.lemma, lex.lemma, lexeme.pos, lex.pos ))
                    #f.write(f"{lexeme.lemma}, {lex.lemma} - STEJNÝ typ: {lex.pos} \n")
    
    
tisknuti("TEST: IDENTICKÁ SLOVA LIŠÍCÍ SE VELIKOSTÍ PRVNÍHO PÍSMENE, KTERÁ K SOBĚ NAVZÁJEM NEJSOU NIJAK PŘIPOJENA A MAJÍ STEJNÝ SLOVNÍ DRUH","test2.1", stejny_typ)
tisknuti("TEST: IDENTICKÁ SLOVA LIŠÍCÍ SE VELIKOSTÍ PRVNÍHO PÍSMENE, KTERÁ K SOBĚ NAVZÁJEM NEJSOU NIJAK PŘIPOJENA A NEMAJÍ STEJNÝ SLOVNÍ DRUH","test2.2", ruzny_typ)



In [52]:
for lexeme in lexicon.iter_lexemes():
    if lexeme.lemma == "stodeset":
        print(lexeme.lemma)



jedna
dvacát


testování 2: přídavná jména končící na -ní by měla být vždy připojena k nějakému slovu
    - využívám v misc toho, že je tam unmotivated, tedy není odvozeno od jiného slova

In [ ]:
with open("test2.txt", "w", encoding="utf-8") as f: #kdyz tam dodam with, tak se mi to samo zavre
    f.write("PŘÍDAVNÁ JMÉNA KONČÍCÍ NA -NÍ, KTERÁ BY MĚLA BÝT PŘIPOJENA K JINÝM SLOVŮM, ALE NEJSOU \n")
    f.write("SLOVO \n")
    for lexeme in lexicon.iter_lexemes():
        if lexeme.pos == "ADJ" and lexeme.lemma.endswith("ní"):
            if 'Loanword' in lexeme.feats.keys() and not lexeme.feats.get('Loanword'): #tedy nejsou odvozena od jiného českého slova
                rodic = lexeme.all_parents 
                if len(rodic) == 0:
                    f.write(f"{lexeme.lemma} \n")

    
#potřebuju rozlišit, když to budou slova přejatá z jiného jazyka - lumbální tu nemá co dělat, ale fauní by mělo být u fauna nebo faun (??)

testování 3: číslovková příslovce - udělat slovník na číslovky a přiřazovat je k nim 
    - tohle by se potom dalo udělat i pro přídavná čísla??

In [43]:
prirazeni = {
    "osedm" : "sedm",
    "pátnáct" : "patnáct",
    "šedesátédevát" : "šedesátýdevátý",
    "tisícát": "tisíc",
}

In [ ]:
def tisk(neexistuje, existuje_delsi):
    with open("test3.txt", "w", encoding="utf-8") as f: #kdyz tam dodam with, tak se mi to samo zavre
        f.write("ČÍSLOVKOVÁ PŘÍSLOVCE, KTERÁ NEJSOU K NIČEMU PŘIPOJENA")
        f.write("\n")
        f.write("EXISTUJE PŘÍDAVNÉ JMÉNO, KE KTERÉMU SE MOHOU PŘIPOJIT:\n")
        f.write(f"{'PŘÍSLOVCE'.ljust(20)}{'PŘÍDAVNÉ JMÉNO'.ljust(20)}\n")
        f.write("\n")
        for adv, adj in sorted(existuje_delsi):
            f.write(f"{adv.ljust(20)}{adj.ljust(20)}\n")
        
        f.write("\n")
        f.write("NEEXISTUJE JEDNO PŘÍDAVNÉ JMÉNO, KE KTERÉMU BY SE TO DALO PŘIPOJIT \n")
        for lemma, rodice in sorted(neexistuje):
            f.write(f"{lemma.ljust(20)}")
            if rodice:
                f.write("\n   možní rodiče: ")
                f.write(", ".join(rodice))
                f.write("\n")
            else:
                f.write(f"nezařazeno \n")

existuje_delsi = set()
neexistuje = []
all_lemmas = {lex.lemma for lex in lexicon.iter_lexemes()}

for lexeme in lexicon.iter_lexemes():
    if lexeme.pos == "ADV" and lexeme.parent is None and "unmotivated" not in lexeme.misc and lexeme.lemma.endswith("é"):
        #a teď mám několik verzí, co by to mohlo být 
        if lexeme.lemma.startswith("propo"):
            slovo = lexeme.lemma[3:]
        elif lexeme.lemma.startswith(("napo", "po", "za", "na")):
            slovo = lexeme.lemma[2:]
        else:
            slovo = lexeme.lemma

        if slovo[:-1] + "ý" in all_lemmas:
                existuje_delsi.add((lexeme.lemma, slovo[:-1] + "ý"))
        elif slovo in all_lemmas and slovo != lexeme.lemma:
             existuje_delsi.add((lexeme.lemma, slovo))
        else:
                if lexeme.lemma.startswith("propo"):
                    slovo = lexeme.lemma[5:]
                elif lexeme.lemma.startswith("napo"):
                    slovo = lexeme.lemma[4:]
                    
                kandidat = []
                volny = []
                for k in prirazeni.keys():
                    if slovo[:-1] == k:
                        kandidat.append(prirazeni[k])

                
                neexistuje.append((lexeme.lemma, kandidat))
                    
tisk(neexistuje,existuje_delsi)


            

osedmé
osedm
pátnácté
pátnáct
tisícáté
tisícát
té
t
šedesátédeváté
šedesátédevát
séparé
sépar


testování 4: přídavná jména končící na 'istický' by měla být připojena k 'ismus' podstatné jméno

In [36]:
def tisk(vysledek, neexistuje):
    with open("test4.txt", "w", encoding="utf-8") as f:
        f.write("PŘÍDAVNÁ JMÉNA KONČÍCÍ NA -ISTICKÝ, KTERÁ NEJSOU K NIČEMU PŘIPOJENA \n")
        f.write("PŘÍDAVNÉ JMÉNO A K ČEMU BY TO MĚLO BÝT PŘIPOJENO\n")
        f.write(f"{'PŘÍDAVJNÉ JMÉNO'.ljust(30)}{'PODSTATNÉ JMÉNO'.ljust(30)}\n")
        f.write("\n")

        for i in vysledek:
            f.write(f"{i[0].ljust(30)}{i[1].ljust(30)}\n")     

        f.write("\n")
        f.write("PŘÍDAVNÉ JMÉNO, K NIMŽ NEBYL NALEZEN PŘEDEK\n")
        f.write(f"PŘÍDAVNÉ JMÉNO \n")
        for i in neexistuje:
            f.write(f"{i} \n")



vysledek = []
neexistuje = []

for lexeme in lexicon.iter_lexemes():
    if lexeme.lemma.endswith("istický") and lexeme.parent is None:
        if not lexeme.lemma.endswith("statistický") and not lexeme.lemma.startswith("proti"):
            nove = lexeme.lemma[:-6]
            nove = nove + "smus"
            if nove in all_lemmas:
                vysledek.append((lexeme.lemma, nove))
            else:
                neexistuje.append(lexeme.lemma)

        if lexeme.lemma.endswith("statistický"):
            vysledek.append((lexeme.lemma, "statistika"))
        
        if lexeme.lemma.startswith("proti"):

            nove = lexeme.lemma[:-6][5:]
            nove = nove + "smus"
            if nove in all_lemmas:
                vysledek.append((lexeme.lemma, nove))
            else:
                neexistuje.append(lexeme.lemma)

tisk(vysledek, neexistuje)



testování 5: slova, které nejsou k ničemu připojena, ale jsou podslovem jiných (adamantin - adaamantinoma)
    - tenhle test musím nechat doběhnout až budu vařit

In [40]:
def tisk(seznam):
    with open("test5.txt", "w", encoding="utf-8") as f:
        f.write("SLOVA, KTERÁ JSOU PODMNOŽINOU JINÝCH SLOV, ALE NEJSOU K SOBĚ NIJAK NAVÁZANÁ \n")
        f.write("UKÁZKA TISKU:\n")
        f.write("spředek \n")
        f.write(f"  slovo, které obsahuje předka \n")
        f.write("\n")

        for slovo, deti in seznam:
            f.write(f"{slovo}\n")
            for m in deti:
                f.write(f"  {m}\n")

def lezeme_nahoru(dite, hledany):
    rodice = dite.all_parents
    if not rodice:
        return False  # nedošli jsme k hledanému
    if hledany in rodice:
        return True  # hledaný je přímo rodičem
    for r in rodice:
        if lezeme_nahoru(r, hledany):  # rekurzivně jdeme dál
            return True
    return False  # pokud jsme ho nikdy nenašli

lemma_to_lex = {lex.lemma: lex for lex in lexicon.iter_lexemes()}
mnozina = []

for lexeme in lexicon.iter_lexemes():
    if lexeme.lemma == "super":
        slovicka = []
        kandidati = {lemma for lemma in all_lemmas if lexeme.lemma in lemma and len(lexeme.lemma) < len(lemma)}
        #najdu všechny slova, ve kterých je lexeme podstringem
        if len(kandidati) > 0: 
            for lemma in kandidati:
                s= lemma_to_lex[lemma]
                dobry = True
                for par in s.all_parents: #a podívám se, jestli se lexeme nenachází schované nad nějakým rodičem dítětě
                    if par != lexeme: #když to samotné není předek
                        predek = lezeme_nahoru(s, lexeme) #tak se podíváme, jestli to není předek předka
                        if predek: #jestli to je předek předka, tak tam nějaké propojení je
                            dobry = False

                    if dobry:
                        slovicka.append(lemma)
        
        if slovicka:
            mnozina.append((lexeme.lemma, slovicka))


# seřazení podle počtu dětí
mnozina = sorted(mnozina, key=lambda x: len(x[1]), reverse=True)
tisk(mnozina)


    

testování 6: když to končí na 'čin' a 'ův', tak to musí být napojeno na mužské a ženské podstatné jméno 
    - 6.1 - ženský rod

In [49]:
def tisk_koncovky(spatny_rod, dobry_rod, spatny_nepripojeno, bez, rod, koncovka, soubor):
    with open(f"{soubor}.txt", "w", encoding="utf-8") as f:
        f.write(f"PŘÍDAVNÁ JMÉNA KONČÍCÍ NA -{koncovka}, KTERÁ NEJSOU PŘIPOJENA KE SLOVU {rod} RODU \n")
        f.write(f"{'PŘÍDAVNÉ JMÉNO'.ljust(20)}{'PODSTATNÉ JMÉNO'.ljust(20)}\n")
        f.write("\n")
        f.write(f"JE PŘIPOJENO KE SLOVU, KTERÉ NENÍ {rod} RODU\n")
        for i in spatny_rod:
            f.write(f"{i[0].ljust(20)}{i[1].ljust(20)}\n")
        f.write("\n")
        f.write(f"NENÍ PŘIPOJENO K PŘEDKOVI, KTERÝ JE {rod} RODU \n")
        for i in dobry_rod:
            f.write(f"{i[0].ljust(20)}{i[1].ljust(20)}\n")
        f.write(f"EXISTUJE NEPŘIPOJENÝ PŘEDEK, KTERÝ NENÍ {rod} RODU\n")
        for i in spatny_nepripojeno:
            f.write(f"{i[0].ljust(20)}{i[1].ljust(20)}\n")
        f.write("NEBYL NALEZEN PŘEDEK \n")
        for i in bez:
            f.write(f"{i} \n")


spatny_rod, dobry_rod, spatny_nepripojeno, bez = [], [], [], []#špatný rod - předek není ženský 
#dobrý rod - není připojen, ale je ženský, spatny_nepripojeno - nepřipojený předek a ještě není ženský
# bez - bez predka
for lexeme in lexicon.iter_lexemes():
    if lexeme.lemma.endswith("čin") and lexeme.pos == "ADJ":
        #tady je to dobře, protože to má být hned nad dítětem
        rodice = lexeme.all_parents
        jeden = False # jestli jsme našli nějakou předkyni
        jiny = False # pokud jsme ale nasli predka, co neni zensky, ale sedí do popisu
        slovo = None
        for r in rodice:
            if r.lemma[:-2] == lexeme.lemma[:-3]:
                jiny = True 
                if r.pos == "NOUN" and r.feats["Gender"] == "Fem":
                    jeden = True
                else: 
                    slovo = r.lemma

        if not jeden and jiny: #máme rodiče, ale není to ženský rod
            spatny_rod.append((lexeme.lemma, slovo))

        elif not jeden and not jiny:
            volny = True
            for lex in lexicon.iter_lexemes():
                if lex.lemma[:-2] == lexeme.lemma[:-3] and lex.lemma != lexeme.lemma and lex.lemma.endswith("a"):
                    if lex.pos == "NOUN" and lex.feats["Gender"] == "Fem":
                        dobry_rod.append((lexeme.lemma, lex.lemma))
                    else:
                        spatny_nepripojeno.append((lexeme.lemma, lex.lemma))
                    volny = False
                    break
            if volny: 
                bez.append(lexeme.lemma)

                
tisk_koncovky(spatny_rod, dobry_rod, spatny_nepripojeno, bez, "ŽENSKÉHO", "ČIN", "test6.1")
                    

KeyboardInterrupt: 

testování 6.2: teď pro muže 

In [ ]:
def tvorba_predka(slovo):
    temp = slovo[:-2]
    nove = temp[0:-1]
    nove2 = temp[-1]
    slovicko = nove + "e" + nove2
    return slovicko
    
def hledani_predka(lexeme):
    rodice = lexeme.all_parents
    pomocnik = tvorba_predka(lexeme.lemma)
    jeden = False # jestli jsme našli nějakou předkyni
    jiny = False # pokud jsme ale nasli predka, co neni muzsky, ale sedí do popisu
    slovo = None
    for r in rodice:
        re = r.lemma
        if r.lemma[-1] in "aeiouyáéíóúůý" and r.lemma[-2] not in "aeiouyáéíóúůý":
            re = r.lemma[:-1]
        if re.endswith("os"):
            re = r.lemma[:-2]
        if r and (re == pomocnik or re == lexeme.lemma[:-2]):
            jiny = True 
            if r.pos == "NOUN" and "Gender" in r.feats.keys() and r.feats["Gender"] == "Masc":
                jeden = True
                break
            else: 
                slovo = r.lemma

    return jeden, jiny, slovo


spatny_rod, dobry_rod, spatny_nepripojeno, bez = [], [], [], []#špatný rod - předek není ženský 
#dobrý rod - není připojen, ale je ženský, spatny_nepripojeno - nepřipojený předek a ještě není ženský
# bez - bez predka

for lexeme in lexicon.iter_lexemes():
    if lexeme.lemma.endswith("ův") and lexeme.pos == "ADJ": #and not test_samohlasek(lexeme.lemma[:-3]):
        #tady je to dobře, protože to má být hned nad dítětem
        #teď mám posichrováno, pokud je to tvaru 'aristokrat > aristokratův'
        jeden, jiny, slovo = hledani_predka(lexeme)
    
        if not jeden and jiny:
            spatny_rod.append((lexeme.lemma, slovo))
        elif not jeden and not jiny:

            pomocnik = tvorba_predka(lexeme.lemma)
            lex = None
            if pomocnik in all_lemmas:
                lex = lexicon.get_lexemes(pomocnik)[0]
            elif lexeme.lemma[:-2] in all_lemmas:
                lex = lexicon.get_lexemes(lexeme.lemma[:-2])[0]

            if lex is not None:
                if 'Gender' in lex.feats.keys() and lex.feats["Gender"] == "Masc":
                    dobry_rod.append((lexeme.lemma, lex.lemma))
                else:
                    spatny_nepripojeno.append((lexeme.lemma, lex.lemma))
            else:
                bez.append(lexeme.lemma)

tisk_koncovky(spatny_rod, dobry_rod, spatny_nepripojeno, bez, "MUŽSKÉHO", "ŮV", "test6.2")

testování 7: tvary jako 'čtyřka' má být u čtyři a podobně

In [51]:
prevod = {
    "jednička" : "jedna",
    "dvojka" : "dva", 
    "trojka" : "tři",
    "čtyřka": "čtyři",
    "pětka": "pět",
    "šestka" : "šest",
    "sedmička" : "sedm",
    "osmička" : "osm",
    "devítka" : "devět", 
    "desítka" : "deset"
}

In [ ]:
def vypisovani(seznamik):
    with open("test7.txt", "w", encoding="utf-8") as f:
        f.write("PODSTATNÁ JMÉNA ODVOZENÁ OD ČÍSLOVEK, ALE NEJSOU K NIM PŘIPOJENA \n")
        f.write("PŘÍKLAD TISKU: \n")
        f.write("podstatné jméno \n")
        f.write(f"  číslovka \n")
        f.write("\n")

        for pod, cisla in seznamik:
            f.write(f"{pod}\n")
            for i in cisla:
                f.write(f"  {i}\n")

seznam = []
for lexeme in lexicon.iter_lexemes():
    rodinka = []
    if lexeme.lemma in prevod.keys(): #tak je tam prostě to slovo (není to dvacetjednička, ale jenom jednička)
        #tohle je prostě pro věci jako je jednička a jedna
        for k in prevod.keys(): 
            if k == lexeme.lemma: 
                rodice = []
                for i in lexeme.all_parents:
                    rodice.append(i.lemma)

                if prevod[k] not in rodice: 
                    rodinka.append(prevod[k])

    elif lexeme.lemma.endswith("náctka"): 
        #patnáctka a tak
        rodic = lexeme.lemma[:-2]
        for i in lexicon.iter_lexemes():
            if i.lemma == rodic and i not in lexeme.all_parents:
                rodinka.append(i.lemma)
    
    elif lexeme.lemma.endswith("ka") and not lexeme.lemma.endswith("strojka"):
        for k in prevod.keys(): 
            if k in lexeme.lemma:
                cast = lexeme.lemma.removesuffix(k)
                rodice = []
                for i in lexeme.all_parents:
                    rodice.append(i.lemma)
                if prevod[k] not in rodice: 
                    rodinka.append(prevod[k])

                if cast != "super":
                    for lex in lexicon.iter_lexemes():
                        #chci, aby pro stošestka se mi jako rodič našlo stošest a ne sto a šest
                        #tak ne, je to rozděleně -> sto šest
                        if lex.lemma == cast and cast not in rodice:
                            rodinka.append(cast)
                            break

    if rodinka:
        seznam.append((lexeme.lemma, rodinka))

vypisovani(seznam)